# Redefining Geographies for Custom Geographies in Replcia

**What I have:** already defined searate census tracts in replica's saved selections, one for each station. 
</br>
**What I need:** combined geographies for all tracts from the already defined stations except for the station listed as origin

In [1]:
import geopandas as gpd
import pandas as pd
from siuba import *
import numpy as np

from calitp_data_analysis.sql import to_snakecase
from calitp_data_analysis import utils
from calitp_data_analysis import get_fs
import gcsfs as fs
fs = get_fs()

import os
import shutil

In [2]:
import replica_utils

In [3]:
gcs_path = "gs://calitp-analytics-data/data-analyses/big_data/lossan_stations/"

In [4]:
station_list_lossan = ["lossan-simi-valley", "lossan-burbank-airport", "lossan-oxnard",  "lossan-camarillo", "lossan-moorpark",  "lossan-chatsworth",
                "lossan-northridge", "lossan-van-nuys", "lossan-burbank", "lossan-glendale", "lossan-union-station",
                "lossan-commerce", "lossan-norwalk", "lossan-buena-park",
                "lossan-fullerton", "lossan-anaheim",  "lossan-orange", "lossan-santa-ana", "lossan-tustin",
                "lossan-irvine", "lossan-laguna-niguel", "lossan-san-juan-capistrano", "lossan-san-clemente", "lossan-san-clemente-pier",
                "lossan-oceanside", "lossan-solana-beach", "lossan-san-diego", "lossan-ot-san-diego"]

In [5]:
station_list_riverside = ["riverside-line-montebello-commerce", "riverside-line-industry", "riverside-line-pomona",  "riverside-line-ontario",
                          "riverside-line-pedley",  "riverside-line-riverside", "lossan-union-station"]

In [6]:
station_list_sb = ["lossan-union-station", "sb-line-cal-state-la", "sb-line-el-monte",  "sb-line-baldwin-park",
                   "sb-line-covina",  "sb-line-pomona-north",
                         "sb-line-claremont", "sb-line-montclair", "sb-line-upland", "sb-line-rancho-cucamonga", "sb-line-fontana", 
                  "sb-line-rialto", "sb-line-san-bernardino-depot", "sb-line-san-bernardino-dt"]


In [7]:
all_stations_list = [ "lossan-simi-valley", "lossan-burbank-airport", "lossan-oxnard",  "lossan-camarillo", "lossan-moorpark",  "lossan-chatsworth",
                     "lossan-northridge", "lossan-van-nuys", "lossan-burbank", "lossan-glendale", 
                     "lossan-commerce", "lossan-norwalk", "lossan-buena-park", "lossan-fullerton", "lossan-anaheim",
                     "lossan-orange", "lossan-santa-ana", "lossan-tustin", "lossan-irvine", "lossan-laguna-niguel", 
                     "lossan-san-juan-capistrano", "lossan-san-clemente", "lossan-san-clemente-pier", "lossan-oceanside", 
                     "lossan-solana-beach", "lossan-san-diego", "lossan-ot-san-diego", "lossan-union-station",
                     
                     "riverside-line-montebello-commerce", "riverside-line-industry", "riverside-line-pomona",
                     "riverside-line-ontario","riverside-line-pedley",  "riverside-line-riverside",
                     
                     "sb-line-cal-state-la", "sb-line-el-monte",  "sb-line-baldwin-park", "sb-line-covina",  "sb-line-pomona-north",
                     "sb-line-claremont", "sb-line-montclair", "sb-line-upland", "sb-line-rancho-cucamonga", "sb-line-fontana", 
                     "sb-line-rialto", "sb-line-san-bernardino-depot", "sb-line-san-bernardino-dt", 
                     "sb-line-redlands-dt", "sb-line-redlands-esri", "sb-line-redlands-univ", "sb-line-san-bernardino-tippecanoe",
                     
                     "91-perris-line-corona", "91-perris-line-corona-west", "91-perris-line-moreno-valley-march", 
                     "91-perris-line-perris-south", "91-perris-line-perris-downtown", "91-perris-line-placentia-future", 
                     "91-perris-line-riverside-hunter-park", "91-perris-line-riverside-la-sierra",
                     
                     "antelope-valley-burbank-airport-n", "antelope-valley-lancaster", "antelope-valley-newhall", "antelope-valley-palmdale",
                     "antelope-valley-santa-clarita", "antelope-valley-sun-valley", "antelope-valley-sylmar-san-fernando",
                     "antelope-valley-via-princessa", "antelope-valley-vincent-grade", "antelope-valley-vista-canyon",
                     
                     
                     "inland-empire-oc-anaheim-canyon", 
                     
                     "ventura-line-ventura-east", "ventura-line-ventura-dtwn",
                     
                    ]
                     

In [8]:
all_stations = replica_utils.import_stations(all_stations_list)

In [9]:
all_stations

,geoid,geoname,alias,centrlat,centrlon,geometry,station_name
0,3612448,"83.02 (Ventura, CA)",None,34.2763,-118.6730,"POLYGON ((-118.68274 34.28135, -118.68274 34.2...",lossan-simi-valley
1,3632207,"83.07 (Ventura, CA)",None,34.2767,-118.6870,"POLYGON ((-118.69138 34.28149, -118.69138 34.2...",lossan-simi-valley
2,3633654,"80.02 (Ventura, CA)",None,34.2687,-118.7266,"POLYGON ((-118.74785 34.26616, -118.74786 34.2...",lossan-simi-valley
3,3646274,"83.04 (Ventura, CA)",None,34.2672,-118.6585,"POLYGON ((-118.67636 34.26690, -118.67635 34.2...",lossan-simi-valley
4,3653763,"82.01 (Ventura, CA)",None,34.2768,-118.7045,"POLYGON ((-118.71775 34.28164, -118.71775 34.2...",lossan-simi-valley
...,...,...,...,...,...,...,...
492,3646035,"23.01 (Ventura, CA)",None,34.2905,-119.2995,"POLYGON ((-119.30352 34.28136, -119.30336 34.2...",ventura-line-ventura-dtwn
493,3670973,"95 (Ventura, CA)",None,34.3650,-119.3572,"MULTIPOLYGON (((-119.44606 34.34783, -119.4460...",ventura-line-ventura-dtwn
494,3671720,"24 (Ventura, CA)",None,34.2765,-119.2945,"POLYGON ((-119.30858 34.28136, -119.30868 34.2...",ventura-line-ventura-dtwn
495,3672846,"22 (Ventura, CA)",None,34.2996,-119.2884,"POLYGON ((-119.30075 34.28104, -119.30051 34.2...",ventura-line-ventura-dtwn


In [10]:
# all_stations>>filter(_.station_name.str.contains("union"))

In [11]:
# all_stations_sb = replica_utils.import_stations(station_list_sb)

# all_stations_riverside = replica_utils.import_stations(station_list_riverside)

In [12]:
all_stations.explore()

In [13]:
all_stations.sample()

,geoid,geoname,alias,centrlat,centrlon,geometry,station_name
49,3634573,"1272.10 (Los Angeles, CA)",None,34.2083,-118.4599,"POLYGON ((-118.46620 34.21330, -118.46620 34.2...",lossan-van-nuys


In [14]:
### combine destination locations, aka all stations but the one at the origin 

In [33]:
# def download_station_geom(df_all_stations, output_type, origin_stations_list, output_name):
#     """
#     this function puts all the functions together, so you dont have to run the commands individually
#     if you want to run only one station or all stations 
    
#     output_type: 'all_stations', 'by_station', 'by_line'
#     """
#     if (output_type == ("all_stations")):
#         df_all_stations_deduplicated = df_all_stations.drop_duplicates(subset=['geoname'], keep='first')
#         utils.make_zipped_shapefile(df_all_stations_deduplicated, f"station_destinations/{output_name}_all_destinations.zip")
        
#         return print("done")
        
#     """
#     aggregates all stations for the destinations with one station as the origin.
#     """
#     elif (output_type == ("by_station")):
        
#         replica_utils.aggregate_destination_station_geometries(df_all_stations, origin_stations_list)
        
#         return print("done")
    
#     """
#     puts the origins as the list of locations and the rest as destinations. NOT aggregating the just the stations specified 
#     """
#     elif (output_type == ("by_line")):
#         df_all_station_line_destinations = df_all_stations[~df_all_stations['station_name'].isin(origin_stations_list)]
#         df_all_station_line_destinations = df_all_station_line_destinations.drop_duplicates(subset=['geoname'], keep='first')
#         utils.make_zipped_shapefile(df_all_station_line_destinations, f"_destinations/{output_name}_all_destinations.zip")
        
        
#     else: 
#         print("Please specify output type")
    
    

In [22]:
# ### create zipped file for all stations

# all_stations_deduplicated = all_stations.drop_duplicates(subset=['geoname'], keep='first')

# utils.make_zipped_shapefile(all_stations_deduplicated, f"station_destinations/metrolink_all_destinations_2.zip")


In [24]:
# #### get all destinations aggregated for ALL SAN BERNARDINO LINE STATIONS
# station_destinations_non_sb = all_stations[~all_stations['station_name'].isin(station_list_sb)]

# station_destinations_non_sb = station_destinations_non_sb.drop_duplicates(subset=['geoname'], keep='first')

# utils.make_zipped_shapefile(station_destinations_non_sb, f"station_destinations/metrolink_all_destinations_sb_origins_2.zip")


In [25]:
# #### get all destinations aggregated for ALL RIVERSIDE LINE STATIONS
# station_destinations_non_riverside = all_stations[~all_stations['station_name'].isin(station_list_riverside)]

# station_destinations_non_riverside = station_destinations_non_riverside.drop_duplicates(subset=['geoname'], keep='first')

# utils.make_zipped_shapefile(station_destinations_non_riverside, f"station_destinations/metrolink_all_destinations_riverside_origins_2.zip")


In [34]:
#### get all destinations aggregated for INDV SAN BERNARDINO LINE STATIONS

# replica_utils.aggregate_destination_station_geometries(all_stations, station_list_sb)

In [35]:
#### get all destinations aggregated for INDV RIVERSIDE LINE STATIONS

# replica_utils.aggregate_destination_station_geometries(all_stations, station_list_riverside)

In [17]:
all_stations_list = [ "lossan-simi-valley", "lossan-burbank-airport", "lossan-oxnard",  "lossan-camarillo", "lossan-moorpark",  "lossan-chatsworth",
                     "lossan-northridge", "lossan-van-nuys", "lossan-burbank", "lossan-glendale", 
                     "lossan-commerce", "lossan-norwalk", "lossan-buena-park", "lossan-fullerton", "lossan-anaheim",
                     "lossan-orange", "lossan-santa-ana", "lossan-tustin", "lossan-irvine", "lossan-laguna-niguel", 
                     "lossan-san-juan-capistrano", "lossan-san-clemente", "lossan-san-clemente-pier", "lossan-oceanside", 
                     "lossan-solana-beach", "lossan-san-diego", "lossan-ot-san-diego",
                     # "lossan-union-station",
                     
#                      "riverside-line-montebello-commerce", "riverside-line-industry", "riverside-line-pomona",
#                      "riverside-line-ontario","riverside-line-pedley",  "riverside-line-riverside",
                     
#                      "sb-line-cal-state-la", "sb-line-el-monte",  "sb-line-baldwin-park", "sb-line-covina",  "sb-line-pomona-north",
#                      "sb-line-claremont", "sb-line-montclair", "sb-line-upland", "sb-line-rancho-cucamonga", "sb-line-fontana", 
#                      "sb-line-rialto", "sb-line-san-bernardino-depot", "sb-line-san-bernardino-dt", 
#                      "sb-line-redlands-dt", "sb-line-redlands-esri", "sb-line-redlands-univ", "sb-line-san-bernardino-tippecanoe",
                     
                     "91-perris-line-corona", "91-perris-line-corona-west", "91-perris-line-moreno-valley-march", 
                     "91-perris-line-perris-south", "91-perris-line-perris-downtown", "91-perris-line-placentia-future", 
                     "91-perris-line-riverside-hunter-park", "91-perris-line-riverside-la-sierra",
                     
                     "antelope-valley-burbank-airport-n", "antelope-valley-lancaster", "antelope-valley-newhall", "antelope-valley-palmdale",
                     "antelope-valley-santa-clarita", "antelope-valley-sun-valley", "antelope-valley-sylmar-san-fernando",
                     "antelope-valley-via-princessa", "antelope-valley-vincent-grade", "antelope-valley-vista-canyon",
                     
                     
                     "inland-empire-oc-anaheim-canyon", 
                     
                     "ventura-line-ventura-east", "ventura-line-ventura-dtwn",
                     
                    ]
                     

In [18]:
len(all_stations_list)

48

In [20]:
### Get all destinations aggregated for ALL individual stations 
# replica_utils.aggregate_destination_station_geometries(all_stations, all_stations_list)

In [25]:
### test worked!
# shp_test = gpd.read_file("origin_antelope-valley-burbank-airport-n_destinations_network_wide.zip")
  

In [26]:
# shp_test.explore()

In [37]:
# shp_test>>count(_.geoname)>>filter(_.n>1)